# Exercise - Regression


The data set for this exercise includes information on house sales in King County, WA (between May 2014 and May 2015). (Each row in the data set pertains to one house. There is a total of 21,613 houses in the data set). Use this data set to predict the sale price of a house (i.e., the `price` column) based on the characteristics of the house. A model can be helpful for buyers, sellers, realtors, and lenders.

## Description of Variables

The description and type of each variable is provided in "KC house data - Data Dictionary.docx". Make sure to read this document to learn about the variables.

## Goal

Use the **kc_house_data.csv** data set and build a model to predict **price**. <br>

# Read and Prepare the Data

In [4]:
# Common imports

import pandas as pd
import numpy as np

np.random.seed(42)

# Get the data

In [5]:
#We will predict the "price" value in the data set:

housing = pd.read_csv("C:\\Users\\hp\\Desktop\\BAIS Semester2\\DSP\\Module2\\kc_house_data.csv")
housing.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,432000,5.0,2.75,2060.0,329903.0,1.5,0,3,5,7.0,2060,0,1989.0,0,zip_98022,47.1776,-121.944,2240,220232.0
1,170000,2.0,1.00,810.0,8424.0,1.0,0,0,4,6.0,810,0,1959.0,0,zip_98023,47.3286,-122.346,820,8424.0
2,235000,3.0,1.00,960.0,5030.0,1.0,0,0,3,7.0,960,0,1955.0,0,zip_98118,47.5611,-122.280,1460,5400.0
3,350000,2.0,1.00,830.0,5100.0,1.0,0,0,4,7.0,830,0,1942.0,0,zip_98126,47.5259,-122.379,1220,5100.0
4,397380,2.0,1.00,1030.0,5072.0,1.0,0,0,3,6.0,1030,0,1924.0,1958,zip_98115,47.6962,-122.294,1220,6781.0


# Split data (train/test)

In [6]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(housing, test_size=0.3)

# Data Prep

Perform your data prep here. You can use pipelines like we do in the tutorials. Otherwise, feel free to use your own data prep steps. Eventually, you should do the following at a minimum:<br>
- Separate inputs from target<br>
- Impute/remove missing values<br>
- Standardize the continuous variables<br>
- One-hot encode categorical variables<br>

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

## Separate the target variable 

In [8]:
train_target = train['price']
test_target = test['price']

train_inputs = train.drop(['price'], axis=1)
test_inputs = test.drop(['price'], axis=1)

## Set the data type of "zipcode" to categorical

In [9]:
train_inputs['zipcode'] = train_inputs['zipcode'].astype('object')
test_inputs['zipcode'] = test_inputs['zipcode'].astype('object')

##  Identify the numeric, binary, and categorical columns

In [10]:
train_inputs.dtypes

bedrooms         float64
bathrooms        float64
sqft_living      float64
sqft_lot         float64
floors           float64
waterfront         int64
view               int64
condition          int64
grade            float64
sqft_above         int64
sqft_basement      int64
yr_built         float64
yr_renovated       int64
zipcode           object
lat              float64
long             float64
sqft_living15      int64
sqft_lot15       float64
dtype: object

In [11]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [12]:
# Identify the binary columns so we can pass them through without transforming
binary_columns = ['waterfront']

In [13]:
# Be careful: numerical columns already includes the binary columns,
# So, we need to remove the binary columns from numerical columns.

for col in binary_columns:
    numeric_columns.remove(col)

In [14]:
binary_columns

['waterfront']

In [15]:
numeric_columns

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15']

In [16]:
categorical_columns

['zipcode']

# Pipeline

In [17]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [18]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [19]:
binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))])

In [20]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        ('binary', binary_transformer, binary_columns)],
        remainder='passthrough')

#passtrough is an optional step. You don't have to use it.

In [21]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        ('binary', binary_transformer, binary_columns)],
        remainder='passthrough')

#passtrough is an optional step. You don't have to use it.

# Transform: fit_transform() for TRAIN

In [22]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

<13724x87 sparse matrix of type '<class 'numpy.float64'>'
	with 233340 stored elements in Compressed Sparse Row format>

In [23]:
train_x.shape

(13724, 87)

# Tranform: transform() for TEST

In [24]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

<5882x87 sparse matrix of type '<class 'numpy.float64'>'
	with 100008 stored elements in Compressed Sparse Row format>

In [25]:
test_x.shape

(5882, 87)

# Calculate the Baseline

In [26]:
from sklearn.dummy import DummyRegressor

dummy_regr = DummyRegressor(strategy="mean")

dummy_regr.fit(train_x, train_y)

NameError: name 'train_y' is not defined

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
# This is the baseline Train RMSE

dummy_train_pred = dummy_regr.predict(train_x)

baseline_train_mse = mean_squared_error(train_y, dummy_train_pred)

baseline_train_rmse = np.sqrt(baseline_train_mse)

print('Baseline Train RMSE: {}' .format(baseline_train_rmse))

In [ ]:
# This is the baseline Test RMSE

dummy_test_pred = dummy_regr.predict(test_x)

baseline_test_mse = mean_squared_error (test_y, dummy_test_pred)

baseline_test_rmse = np.sqrt(baseline_test_mse)

print('Baseline Test RMSE: {}' .format(baseline_test_rmse))

# Train a SGD model (with no regularization)

In [ ]:
from sklearn.linear_model import SGDRegressor 

# eta0 = learning rate
# penalty = regularization term
# max_iter = number of passes over training data (i.e., epochs)

sgd_reg = SGDRegressor(max_iter=100, penalty=None, eta0=0.01) 

sgd_reg.fit(train_x, train_target)



# Predicted vs. Actual values

In [ ]:
sgd_reg.predict(test_x)

In [ ]:
# Create a new DataFrame

predictions = pd.DataFrame(sgd_reg.predict(test_x), columns=['Predicted'])

predictions

In [ ]:
# Add the actual to the same DataFrame

predictions['Actual'] = np.array(test_target)

predictions

### Generate the error metrics

In [ ]:
#Train RMSE
reg_train_pred = sgd_reg.predict(train_x)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))



In [ ]:
#Test RMSE

reg_test_pred = sgd_reg.predict(test_x)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))



# Try L1 Regularization in SGD

In [ ]:
#Stochastic Gradient:

sgd_reg_L1 = SGDRegressor(max_iter=100, penalty='l1', alpha = 0.1, eta0=0.01)

sgd_reg_L1.fit(train_x, train_y)



### Generate the error metrics

In [ ]:
#Train RMSE
reg_train_pred = sgd_reg_L1.predict(train_x)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

In [ ]:
#Test RMSE
reg_test_pred = sgd_reg_L1.predict(test_x)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

In [ ]:
np.random.seed(42)

# tol is the early stopping criteria

sgd_reg_ES = SGDRegressor(max_iter=100, early_stopping = True, n_iter_no_change=5, tol=0.0001, 
                          validation_fraction=0.2, eta0=0.01)

In [ ]:
sgd_reg_L1.coef_

In [ ]:
sgd_reg_L1.n_iter_

In [ ]:
#Train RMSE
reg_train_pred = sgd_reg_L1.predict(train_x)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

In [ ]:
#Test RMSE
reg_test_pred = sgd_reg_L1.predict(test_x)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

# Try L2 Regularization in SGD

In [ ]:

sgd_reg_L2 = SGDRegressor(max_iter=100, penalty='l2', alpha = 20000, eta0=0.01)

sgd_reg_L2.fit(train_x, train_target)


### Generate the error metrics

In [ ]:
#Train RMSE

reg_train_pred = sgd_reg_L2.predict(train_x)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))


In [ ]:
#Test RMSE
reg_test_pred = sgd_reg_L2.predict(test_x)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)


print('Test RMSE: {}' .format(test_rmse))

In [ ]:
np.random.seed(42)

# tol is the early stopping criteria

sgd_reg_ES = SGDRegressor(max_iter=90, early_stopping = True, n_iter_no_change=5, tol=0.0001, 
                          validation_fraction=0.2, eta0=0.01)

In [ ]:
sgd_reg_L2.coef_

In [ ]:
sgd_reg_L2.n_iter_

In [ ]:
#Train RMSE
reg_train_pred = sgd_reg_L2.predict(train_x)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

In [ ]:
#Test RMSE
reg_test_pred = sgd_reg_L2.predict(test_x)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

# Try ElasticNet in SGD

In [ ]:
#Stochastic Gradient:

sgd_reg_elastic = SGDRegressor(max_iter=90, penalty='elasticnet', l1_ratio=0.5, alpha = 10, 
                          eta0=0.01)
sgd_reg_elastic.fit(train_x, train_y)


### Generate the error metrics

In [ ]:
#Train RMSE
reg_train_pred = sgd_reg_elastic.predict(train_x)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

In [ ]:
#Test RMSE
reg_test_pred = sgd_reg_elastic.predict(test_x)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

# Create Polynomial Features

Create polynomial features with degree = 2. 

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# Create second degree terms and interaction terms
poly_features = PolynomialFeatures(degree=2).fit(train_x)

train_x_poly = poly_features.transform(train_x)

test_x_poly = poly_features.transform(test_x)

#This will create the polynomial terms of the categorical variables too

#if degree=3, then it creates all combinations: a, a^2, a^3, b, b^2, b^3, a.b, a^2.b, a.b^2, a^2.b^2 

In [ ]:
#We still fit a linear regression model

pol_lin_reg = SGDRegressor(max_iter=1000, penalty=None, eta0=0.01) 

pol_lin_reg.fit(train_x_poly, train_y)

In [ ]:
train_x_poly.shape

In [ ]:
test_x_poly.shape

In [ ]:
#Train RMSE
reg_train_pred = pol_lin_reg.predict(train_x_poly)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

In [ ]:
#Test RMSE
reg_test_pred = pol_lin_reg.predict(test_x_poly)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

# Try L2 Regularization in SGD (with polynomial features)

In [ ]:

pol_lin_reg = SGDRegressor(max_iter=1000, penalty='l2',alpha = 0.1, eta0=0.01) 

pol_lin_reg.fit(train_x_poly, train_y)

### Generate the error metrics

In [ ]:
poly_reg_pred_train =pol_lin_reg .predict(train_x_poly)

train_mse = mean_squared_error(train_y, poly_reg_pred_train)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

In [ ]:
poly_reg_pred_test =pol_lin_reg .predict(train_x_poly)

train_mse = mean_squared_error(train_y, poly_reg_pred_test)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))